## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Balkhash,KZ,46.8440,74.9804,74.97,49,4,7.31,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,34.48,87,48,2.48,scattered clouds
2,2,Kimbe,PG,-5.5502,150.1429,79.07,82,100,3.31,light rain
3,3,Kabalo,CD,-6.0500,26.9167,75.36,35,34,4.83,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,74.37,83,48,14.52,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kimbe,PG,-5.5502,150.1429,79.07,82,100,3.31,light rain
3,3,Kabalo,CD,-6.0500,26.9167,75.36,35,34,4.83,scattered clouds
8,8,Kapaa,US,22.0752,-159.3190,88.12,70,75,5.01,broken clouds
15,15,Umm Lajj,SA,25.0213,37.2685,87.64,49,3,7.47,clear sky
19,19,Puerto Ayora,EC,-0.7393,-90.3518,82.36,85,47,7.00,scattered clouds
23,23,Avera,US,33.1940,-82.5271,83.44,64,98,10.98,light rain
35,35,Norman Wells,CA,65.2820,-126.8329,78.33,31,53,5.66,broken clouds
37,37,Avarua,CK,-21.2078,-159.7750,78.85,89,75,12.66,light rain
39,39,Acacoyagua,MX,15.3500,-92.6500,81.64,82,99,2.75,overcast clouds
46,46,Portland,US,45.5234,-122.6762,82.45,59,90,5.75,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kimbe,PG,79.07,light rain,-5.5502,150.1429,
3,Kabalo,CD,75.36,scattered clouds,-6.0500,26.9167,
8,Kapaa,US,88.12,broken clouds,22.0752,-159.3190,
15,Umm Lajj,SA,87.64,clear sky,25.0213,37.2685,
19,Puerto Ayora,EC,82.36,scattered clouds,-0.7393,-90.3518,
23,Avera,US,83.44,light rain,33.1940,-82.5271,
35,Norman Wells,CA,78.33,broken clouds,65.2820,-126.8329,
37,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
39,Acacoyagua,MX,81.64,overcast clouds,15.3500,-92.6500,
46,Portland,US,82.45,overcast clouds,45.5234,-122.6762,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

# Display new table with Hotel Names
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kimbe,PG,79.07,light rain,-5.5502,150.1429,Kimbe Bay Hotel
3,Kabalo,CD,75.36,scattered clouds,-6.0500,26.9167,
8,Kapaa,US,88.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Umm Lajj,SA,87.64,clear sky,25.0213,37.2685,Moon light Furnished Units
19,Puerto Ayora,EC,82.36,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
23,Avera,US,83.44,light rain,33.1940,-82.5271,
35,Norman Wells,CA,78.33,broken clouds,65.2820,-126.8329,Heritage Hotel
37,Avarua,CK,78.85,light rain,-21.2078,-159.7750,Paradise Inn
39,Acacoyagua,MX,81.64,overcast clouds,15.3500,-92.6500,Finca TAJUKO
46,Portland,US,82.45,overcast clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kimbe,PG,79.07,light rain,-5.5502,150.1429,Kimbe Bay Hotel
8,Kapaa,US,88.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Umm Lajj,SA,87.64,clear sky,25.0213,37.2685,Moon light Furnished Units
19,Puerto Ayora,EC,82.36,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
35,Norman Wells,CA,78.33,broken clouds,65.2820,-126.8329,Heritage Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))